In [ ]:
import os
import sys
from dotenv import find_dotenv
sys.path.append(os.path.dirname(find_dotenv()))

import numpy as np
import time as time
import glob
import random
from pathlib import Path
import constants

# Set up matplotlib and use a nicer set of plot parameters
%config InlineBackend.rc = {}
import matplotlib as mpl
mpl.rc('mathtext',fontset='stixsans')
mpl.rc('figure', facecolor="white")
#matplotlib.rc_file("../../templates/matplotlibrc")
import matplotlib.pyplot as plt
#import matplotlib.colors as colors
# %matplotlib notebook
import configparser


from astropy.cosmology import FlatLambdaCDM
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord

# Define cosmology
cosmo = constants.COSMOLOGY

PiBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins23_pi_0-30hMpc.txt')
SigBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins10_sigma_0-30hMpc.txt')

PiBins0 = ascii.read(PiBin_fil)
SigBins0 = ascii.read(SigBin_fil)

PiEdges = PiBins0['pi_edges'].data
SigEdges = SigBins0['sigma_edges'].data

# Convert bin boundaries from Mpc/h to Mpc
PiEdges  = PiEdges/(len(PiEdges)*[cosmo.h])
SigEdges = SigEdges/(len(SigEdges)*[cosmo.h])

print('Pi bin edges in Mpc:')
print(PiEdges)
print('Sigma bin edges in Mpc:')
print(SigEdges)

MOCK_XCORR_DIR = os.path.join(constants.BIAS_DIR_BASE, 'xcorr', 'mock', 'crosscorr')
AVG_XCORR_OUTPUT_DIR = os.path.join(constants.BIAS_DIR_BASE, 'xcorr', 'mock', 'crosscorr-avg')

In [ ]:
X, Y = np.meshgrid(SigEdges, PiEdges)

SigMax = 35.
PiMin = -40.
PiMax = 40.

def plot_one_crosscorr(crosscorr_path, xcorr=None, ax=None):
    if xcorr is None:
        xcorr = np.load(crosscorr_path).T

    if ax is None:
        plt.figure(figsize=(2, 14/3))
    plt.pcolormesh(X, Y, xcorr,cmap='jet_r',vmin=-0.2, vmax=0.1 )
    plt.gca().set_aspect('equal')
    plt.gca().set_xlim(np.min(X), SigMax)
    plt.gca().set_ylim(PiMin, PiMax)
    plt.gca().set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
    plt.gca().set_ylabel(r'$\pi\; (\mathrm{cMpc})$')

In [ ]:
# for p in random.sample(list(glob.glob(os.path.join(MOCK_XCORR_DIR, '*.npy'))), 100):
#     plot_one_crosscorr(p)
#     plt.show()

In [ ]:
def average_xcorrs(prefix):
    xcorr_path_list = glob.glob(os.path.join(MOCK_XCORR_DIR, f"{prefix}*"))
    xcorr_plot_sum = 0
    for i, p in enumerate(xcorr_path_list):
        xcorr_plot_sum += np.load(p).T
    return xcorr_plot_sum / len(xcorr_path_list)

In [ ]:
# Read bin edges
PiBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins23_pi_0-30hMpc.txt')
SigBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins10_sigma_0-30hMpc.txt')

PiBins0 = ascii.read(PiBin_fil)['pi_edges'].data
SigBins0 = ascii.read(SigBin_fil)['sigma_edges'].data

# vega interprets these as center values.
RP_VALS = (PiBins0[1:] + PiBins0[:-1]) / 2
RT_VALS =  (SigBins0[1:] + SigBins0[:-1]) / 2

RT_GRID, RP_GRID = np.meshgrid(RT_VALS, RP_VALS)

Z_EFF = 2.3

VEGA_TEMPLATE_DIR = Path('/global/homes/b/bzh/clamato-xcorr/bias-err-pipeline')

survey_param_dict = {
    'MOSDEF': (195, -1.12, 1.5),
    'zDeep': (759, -3.04, 2),
    # 'VUDS': (469, -2.65, dkms_to_dmpch(200)),
    'CLAMATO': (205, -1.27, 1.5),
    # '3DHST': (322, -1, dz_to_dmpch(0.0034 * (1 + zmid)))
}

output_base = Path(AVG_XCORR_OUTPUT_DIR)
survey_name = 'CLAMATO'
fit_dispersion = False

# Case-insensitive glob: https://stackoverflow.com/a/10886685
def insensitive_glob(pattern):
    def either(c):
        return '[%s%s]' % (c.lower(), c.upper()) if c.isalpha() else c
    return glob.glob(''.join(map(either, pattern)))

def gen_vega_config_and_data(xcorr_path,
                             use_raw_covar=True):
    
    suffix = Path(xcorr_path).name.replace('xcorrmock', '').replace('.npy', '')
    assert survey_name in suffix
    
    hdul_path = output_base / f'data{suffix}.fits'

    if True:
        # Generate picca-format FITS file.
        obs_xcorr = np.load(xcorr_path)
        covar_path = insensitive_glob(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'covar', f"covar_{'raw' if use_raw_covar else ''}mock*_"+survey_name+f"_{constants.DATA_VERSION}.npy"))
        assert len(covar_path) == 1
        covar = np.load(covar_path[0])
        assert obs_xcorr.dtype == np.double and covar.dtype == np.double

        data_table = Table([covar, obs_xcorr.flatten(), RT_GRID.flatten(order='F'), RP_GRID.flatten(order='F'), np.full(obs_xcorr.size, Z_EFF)],
                          names=['CO', 'DA', 'RT', 'RP', 'Z'])
        data_hdu = fits.BinTableHDU(data=data_table)
        data_hdu.header['RPMIN'] = min(RP_VALS)
        data_hdu.header['RPMAX'] = max(RP_VALS)
        data_hdu.header['NP'] = len(RP_VALS)
        data_hdu.header['RTMIN'] = min(RT_VALS)
        data_hdu.header['RTMAX'] = max(RT_VALS)
        data_hdu.header['NT'] = len(RT_VALS)
        data_hdu.header['BLINDING'] = 'none'
        hdul = fits.HDUList([fits.PrimaryHDU(), data_hdu])
        hdul.writeto(hdul_path, overwrite=True)
    
    # Copy template .ini files to output base folder, and change filenames.
    tracer_cfg = configparser.ConfigParser()
    tracer_cfg.optionxform=str
    tracer_cfg.read(VEGA_TEMPLATE_DIR / 'bias_qsoxlya_template.ini')
    tracer_cfg['data']['filename'] = str(hdul_path)
    tracer_cfg_path = output_base / f'bias_qsoxlya{suffix}.ini'
    with open(tracer_cfg_path, 'w') as f:
        tracer_cfg.write(f)
    
    main_cfg = configparser.ConfigParser()
    main_cfg.optionxform=str
    main_cfg.read(VEGA_TEMPLATE_DIR / 'bias_main_template.ini')
    main_cfg['data sets']['ini files'] = str(tracer_cfg_path)
    
    # Input fixed parameters here.
    main_cfg['parameters']['drp_QSO'] = str(survey_param_dict[survey_name][1])
    main_cfg['parameters']['sigma_velo_disp_gauss_QSO'] = str(survey_param_dict[survey_name][2] * 2)
    if fit_dispersion:
        main_cfg['sample']['sigma_velo_disp_gauss_QSO'] = 'True'
        
    with open(output_base / f'bias_main{suffix}.ini', 'w') as f:
        main_cfg.write(f)

In [ ]:
log_lb_list = np.arange(9, 13.4 - 0.1, 0.1)

avg_xcorr_list = []

for lb in log_lb_list:
    ub = lb + 0.1
    input_str = f'xcorrmock_survey_{survey_name}_mass_{lb}_{ub}'
    
    avg_xcorr = average_xcorrs(input_str)
    avg_output_path = os.path.join(AVG_XCORR_OUTPUT_DIR, f'{input_str}_avg.npy')
    np.save(avg_output_path, avg_xcorr.T)
    gen_vega_config_and_data(avg_output_path)

    fig, axes = plt.subplots(1, 2, figsize=(4, 4))
    plt.sca(axes[0])
    plot_one_crosscorr('', xcorr=avg_xcorr, ax=axes[0])
    plt.title(f'{lb:.1f} {ub:.1f}')
    bestfit_model = np.load(os.path.join(AVG_XCORR_OUTPUT_DIR, f'bestfitmodel_survey_{survey_name}_mass_{lb}_{ub}_avg.npy'))
    bestfit_model = bestfit_model.reshape(*avg_xcorr.shape, order='F')
    plt.sca(axes[1])
    plot_one_crosscorr('', xcorr=bestfit_model, ax=axes[1])
    bestfit_params = np.load(os.path.join(AVG_XCORR_OUTPUT_DIR, f'bestfit_survey_{survey_name}_mass_{lb}_{ub}_avg.npz'))
    plt.title(f'Chisq: {bestfit_params["reduced_chisq"]:.1f} | Bias: {bestfit_params["bias_QSO"][0]:.2f} | Disp: {bestfit_params["sigma_velo_disp_gauss_QSO"][0]:.2f}')
    plt.show()